# NOAA Marine Pollution Surveillance Reports Web Scraper

This code scrapes salient information from the linked .txt files of the NOAA MSPR service website and saves the output as a JSON file.

In [ ]:
#Import libraries
import requests
import urllib.request
import time
import numpy as np
import pandas as pd
import geopandas
import re
import copy
from bs4 import BeautifulSoup
import sqlite3
from twilio.rest import Client

In [ ]:
# Function to convert DMS to DD; adapted from 
# https://gist.github.com/chrisjsimpson/076a82b51e8540a117e8aa5e793d06ec
def dms2dec(dms_str):
    """Converts coordinates in Degrees Minutes 
    Seconds (DMS) to Decimal Degrees (DD)

    Parameters
    ----------
    dms_str : str
        Coordinates in DMS

    Returns
    -------
    returned_data : str
        Coordinates in DD
    """

    dms_str = re.sub(r'\s', '', dms_str)
    sign = -1 if re.search('[swSW]', dms_str) else 1
    numbers = [*filter(len, re.split('\D+', dms_str, maxsplit=4))]
    degree = numbers[0]
    minute = numbers[1] if len(numbers) >= 2 else '0'
    second = numbers[2] if len(numbers) >= 3 else '0'
    frac_seconds = numbers[3] if len(numbers) >= 4 else '0'
    second += "." + frac_seconds
    return sign * (int(degree) + float(minute) / 60 + float(second) / 3600)

## Compile links to NOAA MPSR reports from yearly report pages

In [ ]:
#URL's for each years web pages from NOAA MPSR
urls = ['https://www.ospo.noaa.gov/Products/ocean/marinepollution/2020_archive.html',
        'https://www.ospo.noaa.gov/Products/ocean/marinepollution/2019_archive.html',
        'https://www.ospo.noaa.gov/Products/ocean/marinepollution/2018_archive.html']

#Iterate through web pages for each year and extract links to 'txt' files for each report
i=0
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #Get the links to MPSR reports
    if i == 0:
        files = soup.findAll('a', string='txt')
        i = i + 1
    else:
        files = files + soup.findAll('a', string='txt')

#TESTING - create subset of specific files using list comprehension
#files = [files[i] for i in (1, 50, 500, 600)]

## Update database

In [ ]:
conn = sqlite3.connect('noaa_mpsr.sqlite')
cur = conn.cursor()

#Drop table for testing purposes
#cur.execute('DROP TABLE IF EXISTS reports')

cur.execute('CREATE TABLE IF NOT EXISTS reports (url TEXT UNIQUE, data TEXT)')
    
new_reports = 0
    
for file in files:
    
    url = 'https://www.ospo.noaa.gov' + file['href']
    
    #Check and skip reports in HTML format
    if '.html' in url: 
        #print('.html file skipped')
        continue

    #Check if row is already in database and add if not
    cur.execute('SELECT url FROM reports WHERE url= ?', (url, ) )
    msg = cur.fetchone()
    
    if not msg:
        
        report_data = requests.get(url)
        report_data = report_data.text
        cur.execute('''INSERT OR IGNORE INTO reports (url, data) VALUES ( ?, ? )''', ( url, report_data) )
        conn.commit()
        
        new_reports = new_reports + 1
        
        #Sleep between file downloads to avoid being blocked
        time.sleep(.10)
        
    else:
        continue

print('There are', new_reports, 'new NOAA MPSR reports')

## Process each report and save into Geojson file

In [ ]:
if new_reports >= 1:

    #Setup dictionary to store MPSR reports data with defined keys
    data_template = {'SOURCE': None,
                     'REPORT DATE': None, 
                     'REPORT TIME': None,
                     'REGION': None,
                     'SUB REGION': None,
                     'AREA/BLOCK': None,
                     'IMAGE DATE': None,
                     'IMAGE TIME': None,
                     'DATA SOURCE': None,
                     'MODE': None,
                     'RESOLUTION': None,
                     'LOCATION': None,
                     'AREA': None,
                     'CONFIDENCE': None,
                     'REMARKS': None,
                     'UNCERTAINTIES': None,
                     'ANALYST': None,                 
                     'ERMA': None}

    #Process each report
    #Filler value
    filler = 'Null'

    #Get the data from the database
    cur.execute('SELECT * FROM reports')
    files = cur.fetchall()

    report_count = 1

    #Process each report in the database
    for file in files:

        #Create new blank temporary dictionary from template
        data_temp = copy.deepcopy(data_template)

        #Split txt file by carriage return
        lines = file[1].splitlines()

        #For each file populate each key/value pair with a placeholder value to ensure arrays are all the same size
        for key in data_temp: data_temp[key] = [filler]        

        #Get source URL from database
        data_temp['SOURCE'] = file[0]

        #Process each line
        line_count = 0
        for line in lines:

            #Check for non standard lines and blank lines
            if '***NOTE' in line: 
                continue
            if not line: 
                continue

            #Split lines by colon
            line_data = line.split(':', 1)
            label = line_data[0]        

            #Check if the current label is a dictionary key
            if label in data_temp:
                #try and except because some files have poorly formed data
                try:
                    row_data = line_data[1]
                    row_data = row_data.strip()
                    data_temp[label] = [row_data]
                except:
                    continue

        #Compile results into dataframe
        if report_count is 1:
            data_table_df = pd.DataFrame.from_dict(data_temp)
        else:
            data_temp_df = pd.DataFrame.from_dict(data_temp)
            data_table_df = data_table_df.append(data_temp_df, ignore_index=True)

        #Print status message
        #print(report_count, 'of', len(files), 'reports processed')

        report_count = report_count + 1


    #Re-format LOCATION field into LAT and LON

    #Create new data frame splitting LOCATION into LAT/LON
    location = data_table_df['LOCATION'].str.split("/", n = 1, expand = True)

    #Merge split LAT/LON back into data_table_df and drop old field
    data_table_df['LAT'] = location[0]
    data_table_df['LON'] = location[1]
    data_table_df.drop(columns =['LOCATION'], inplace = True)

    #Convert DMS to DD
    data_table_df['LAT'] = data_table_df['LAT'].apply(dms2dec)
    data_table_df['LON'] = data_table_df['LON'].apply(dms2dec)

    #Convert to geodataframe
    data_table_gdf = geopandas.GeoDataFrame(
        data_table_df, geometry=geopandas.points_from_xy(data_table_df.LON, data_table_df.LAT))

    #data_table_gdf.style

    #Write to Geojson file
    data_table_gdf.to_file("NOAA_MPSR.geojson", driver='GeoJSON')

else:
    print('No new reports, aborting creation of Geojson')

## Send SMS message to notify when updates have been made

In [ ]:
# Your Account SID from twilio.com/console
account_sid = ''

# Your Auth Token from twilio.com/console
auth_token  = ''

client = Client(account_sid, auth_token)

message_body = 'NOAA MPSR update: there are %s new reports' %(new_reports)

# Add from and to numbers below
if new_reports >= 1:
    message = client.messages.create(
        to='', 
        from_='',
        body=message_body)

    print(message.sid)